In [2]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim
from torch import nn
from tqdm import tqdm
import matplotlib.pyplot as plt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_data = datasets.MNIST(root = 'data',train = True,transform = transforms.ToTensor(), download = True)

test_data = datasets.MNIST(root = 'data',train = False, download = True,transform = transforms.ToTensor())

batch_size = 64

train_loader = DataLoader(train_data,batch_size,shuffle=True)
test_loader = DataLoader(test_data,batch_size,shuffle=True)

class CNN(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    super(CNN,self).__init__()
    self.convl1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1,padding=1)
    self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
    self.convl2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1)
    self.fc1 = nn.Linear(64*7*7,128)
    self.fc2 = nn.Linear(128,num_classes)
  def forward(self,x):
    x = F.relu(self.convl1(x))
    x = self.pool(x)
    x = F.relu(self.convl2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0],-1)
    x = self.fc1(x)
    x = self.fc2(x)
    return x

model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

num_epoch = 10

def accuracy(pred,target):
  correct = torch.eq(pred,target).sum().item()
  acc = (correct/len(target))*100
  return acc

def train_model(model,train_loader,num_epoch,criterion,optimizer,device):

  model.train()
  total_preds = []
  total_targets = []


  for batch_indx ,(data,targets) in enumerate(train_loader):
    data = data.to(device)
    targets = targets.to(device)

    pred_score = model(data)

    train_loss = criterion(pred_score,targets)
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    pred = pred_score.argmax(dim = 1, keepdim = False)
    total_preds.extend(pred.cpu())
    total_targets.extend(targets.cpu())

    if batch_indx % 100 == 0:
      batch_accuracy = accuracy(torch.tensor(total_targets), torch.tensor(total_preds))
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2f}%'.format(
                epoch+1, batch_indx * len(data), len(train_loader.dataset),
                100. * batch_indx / len(train_loader), train_loss.item(), batch_accuracy))
      total_preds.clear()
      total_targets.clear()

def test_model(model,test_loader,criterion,device):

  model.eval()
  total_preds = []
  total_targets = []
  test_loss = 0

  with torch.no_grad():
    for batch_indx ,(data,targets) in enumerate(test_loader):
      data = data.to(device)
      targets = targets.to(device)

      pred_score = model(data)

      test_loss += criterion(pred_score,targets).item()
      pred = pred_score.argmax(dim = 1, keepdim = False)
      total_preds.extend(pred.cpu())
      total_targets.extend(targets.cpu())


    test_loss /= len(test_loader.dataset)
    acc = accuracy(torch.tensor(total_targets), torch.tensor(total_preds))
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, len(total_preds), len(test_loader.dataset),
        acc))
    total_preds.clear()
    total_targets.clear()


for epoch in range(num_epoch):
  train_model(model,train_loader,num_epoch,criterion,optimizer,device)
  test_model(model,test_loader,criterion,device)








Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 9.91M/9.91M [00:00<00:00, 58.4MB/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.98MB/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 1.65M/1.65M [00:00<00:00, 14.6MB/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.93MB/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



Train Epoch: 1 [0/60000 (0%)]	Loss: 2.293964	Accuracy: 10.94%
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.048973	Accuracy: 82.34%
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.322093	Accuracy: 94.69%
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.064281	Accuracy: 96.19%
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.022266	Accuracy: 97.16%
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.074493	Accuracy: 96.78%
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.018104	Accuracy: 97.62%
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.029575	Accuracy: 97.84%
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.049410	Accuracy: 98.00%
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.031051	Accuracy: 97.98%

Test set: Average loss: 0.0009, Accuracy: 10000/10000 (98.14%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.045429	Accuracy: 98.44%
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.045024	Accuracy: 98.00%
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.008254	Accuracy: 98.41%
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.060731	Accuracy: 98.45%
Train Epo